### Introduction

##### Experiment setup:
In this notebook I'm presenting a time benchmark which compares the three options to load images for Image Classification task:
1. `tfdata-image-loader` (my repo) 
2. The default Keras `flow_from_directory` option,
3. The new keras.preprocessing `image_dataset_from_directory`

In order to perform the measurements I'm fine tuning a MobilenetV2 head on a new set of classes.  


##### Experiment assumptions

In order to train MobilenetV2 we want to to the following operations:
* resize to (224, 224), 
* normalize values to -1, 1, 
* randomly flip some images, 
* do not cache the content, 
* prefetch newer samples (if possible).

##### Hardware acceleration
For the sake of the experiment I used a GPU. You can choose to use a GPU in `Runtime > Change runtime type > GPU`.

In [3]:
!nvidia-smi

Mon Sep 14 18:11:51 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.66       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   35C    P8     7W /  75W |      0MiB /  7611MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

### Install TFData Image Loader

In [4]:
!pip install -e git+git://github.com/sebastian-sz/tfdata-image-loader.git#egg=tfdata-image-loader

Obtaining tfdata-image-loader from git+git://github.com/sebastian-sz/tfdata-image-loader.git#egg=tfdata-image-loader
  Cloning git://github.com/sebastian-sz/tfdata-image-loader.git to ./src/tfdata-image-loader
  Running command git clone -q git://github.com/sebastian-sz/tfdata-image-loader.git /content/src/tfdata-image-loader
  Running setup.py develop for tfdata-image-loader


After installing the external module, please restart your runtime.   
Alternatively you can run:

In [ ]:
import os

def restart_runtime():
  os.kill(os.getpid(), 9)

restart_runtime()

### Download and unpack example dataset:

In [1]:
!curl https://storage.googleapis.com/download.tensorflow.org/example_images/flower_photos.tgz |tar xzf -

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  218M  100  218M    0     0  89.1M      0  0:00:02  0:00:02 --:--:-- 89.1M


Remove the license file so it doesn't mess up our directory tree structure:

In [2]:
!rm flower_photos/LICENSE.txt

### Setting up our experiment

We are going to define few constants which are going to be helpful during our comparison:

In [3]:
import os
import pathlib
import time

import numpy as np
import tensorflow as tf

from tfdata_image_loader import TFDataImageLoader

print(f"Is gpu available: {tf.test.is_gpu_available()}")

tf.__version__

Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.
Is gpu available: True


'2.3.0'

In [4]:
EPOCHS = 10
INPUT_SHAPE = (1, 224, 224, 3)

BATCH_SIZE = 32

DATA_PATH = "./flower_photos/"
NUM_CLASSES = len(os.listdir(DATA_PATH))

SEED = 42
np.random.seed(SEED)
tf.random.set_seed(SEED)

IMG_HEIGHT = IMG_WIDTH = INPUT_SHAPE[1]

### Create Data Loaders 

We are going to create two data loaders:
1. Custom module: `tfdata-image-loader`.
2. The default Keras `flow_from_directory`.
3. New Keras preprocessing `image folder dataset`

#### Create TFData Image Loader and load dataset:

In [5]:
def preprocess_data(image, label):
    return (image / 127.5) - 1, label


def augment_data(image, label):
    return tf.image.random_flip_left_right(image), label

In [6]:
train_loader = TFDataImageLoader(
    data_path=DATA_PATH,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    pre_process_function=preprocess_data,
    augmentation_function=augment_data,
)

Found 3670 images, belonging to 5 classes

Class names mapping: 
{'daisy': array([1, 0, 0, 0, 0], dtype=int32), 'dandelion': array([0, 1, 0, 0, 0], dtype=int32), 'roses': array([0, 0, 1, 0, 0], dtype=int32), 'sunflowers': array([0, 0, 0, 1, 0], dtype=int32), 'tulips': array([0, 0, 0, 0, 1], dtype=int32)}



In [7]:
tfdata_image_loader_ds = train_loader.load_dataset()

#### Create Keras Image Loader:

In [8]:
def preprocess_image(img):
    return (img / 127.5) - 1

train_image_generator = tf.keras.preprocessing.image.ImageDataGenerator(
    preprocessing_function=preprocess_image,
    horizontal_flip=True,
)

In [9]:
keras_data_gen = train_image_generator.flow_from_directory(
    directory=DATA_PATH,
    batch_size=BATCH_SIZE,
    shuffle=True,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
)

Found 3670 images belonging to 5 classes.


In [10]:
train_image_count = len(list(pathlib.Path(DATA_PATH).glob('*/*.jpg')))
train_steps_per_epoch = np.ceil(train_image_count/BATCH_SIZE)

#### Keras Preprocessing `image_dataset_from_directory`:

In [11]:
image_dataset_from_directory = tf.keras.preprocessing.image_dataset_from_directory(
  DATA_PATH,
  seed=SEED,
  image_size=(IMG_HEIGHT, IMG_WIDTH),
  batch_size=BATCH_SIZE,
  label_mode="categorical"
  )

normalization_layer = tf.keras.layers.experimental.preprocessing.Rescaling(scale=1./127.5, offset=-1)

Found 3670 files belonging to 5 classes.


In [12]:
image_dataset_from_directory = image_dataset_from_directory.map(lambda x, y: (normalization_layer(x), y)).map(augment_data).prefetch(buffer_size=tf.data.experimental.AUTOTUNE)

# Or try without norm layer (similar results):
# image_dataset_from_directory = image_dataset_from_directory.map(preprocess_data).map(augment_data).prefetch(buffer_size=tf.data.experimental.AUTOTUNE)

### Example model declaration
We are going to use a pretrained MobilenetV2 with a custom head.

In [13]:
def build_model(num_classes):
  base_model = tf.keras.applications.MobileNetV2(
      include_top=False,
      pooling="avg",
      input_shape=INPUT_SHAPE[1:]
  )
  base_model.trainable = False

  return tf.keras.Sequential([
      base_model,
      tf.keras.layers.Dropout(0.2),
      tf.keras.layers.Dense(num_classes, activation="softmax")             
  ])

In [14]:
def build_fresh_model(num_classes):
  model = build_model(num_classes=num_classes)
  model.compile(
      optimizer=tf.keras.optimizers.RMSprop(),
      loss=tf.keras.losses.CategoricalCrossentropy(from_logits=False),
      metrics=["accuracy"]      
  )
  return model

### Warmup runs

We are going to make 3 epochs warm up before we make time measurements:

In [15]:
model = build_fresh_model(NUM_CLASSES)
model.fit(tfdata_image_loader_ds, epochs=3)

9412608/9406464 [==============================] - 0s 0us/step
Epoch 1/3
115/115 [==============================] - 10s 85ms/step - loss: 0.6985 - accuracy: 0.7436
Epoch 2/3
115/115 [==============================] - 10s 83ms/step - loss: 0.3903 - accuracy: 0.8676
Epoch 3/3
115/115 [==============================] - 9s 83ms/step - loss: 0.3001 - accuracy: 0.8967


In [16]:
model = build_fresh_model(NUM_CLASSES)
model.fit(
    keras_data_gen,
    steps_per_epoch=train_steps_per_epoch,
    epochs=3,
    )

Epoch 1/3
115/115 [==============================] - 15s 133ms/step - loss: 0.6926 - accuracy: 0.7450
Epoch 2/3
115/115 [==============================] - 15s 133ms/step - loss: 0.3739 - accuracy: 0.8738
Epoch 3/3
115/115 [==============================] - 15s 133ms/step - loss: 0.3057 - accuracy: 0.8891


In [17]:
model = build_fresh_model(NUM_CLASSES)
model.fit(
    image_dataset_from_directory,
    epochs=3,
    )

Epoch 1/3
115/115 [==============================] - 12s 105ms/step - loss: 0.7051 - accuracy: 0.7452
Epoch 2/3
115/115 [==============================] - 12s 103ms/step - loss: 0.3749 - accuracy: 0.8640
Epoch 3/3
115/115 [==============================] - 12s 105ms/step - loss: 0.3192 - accuracy: 0.8880


### Measure performance

After warm up runs we are going to measure the performance of two loaders:

##### Measure time of my TFData Image Loader 

In [18]:
model = build_fresh_model(NUM_CLASSES)

In [19]:
start = time.perf_counter()
_ = model.fit(tfdata_image_loader_ds, epochs=EPOCHS)

print(f"Train job took: {time.perf_counter() - start} seconds.")

Epoch 1/10
115/115 [==============================] - 9s 79ms/step - loss: 0.7097 - accuracy: 0.7417
Epoch 2/10
115/115 [==============================] - 9s 79ms/step - loss: 0.3682 - accuracy: 0.8719
Epoch 3/10
115/115 [==============================] - 9s 79ms/step - loss: 0.3066 - accuracy: 0.8913
Epoch 4/10
115/115 [==============================] - 9s 77ms/step - loss: 0.2712 - accuracy: 0.9090
Epoch 5/10
115/115 [==============================] - 9s 79ms/step - loss: 0.2435 - accuracy: 0.9147
Epoch 6/10
115/115 [==============================] - 9s 78ms/step - loss: 0.2245 - accuracy: 0.9237
Epoch 7/10
115/115 [==============================] - 9s 77ms/step - loss: 0.2133 - accuracy: 0.9245
Epoch 8/10
115/115 [==============================] - 9s 78ms/step - loss: 0.1932 - accuracy: 0.9338
Epoch 9/10
115/115 [==============================] - 9s 78ms/step - loss: 0.1846 - accuracy: 0.9403
Epoch 10/10
115/115 [==============================] - 9s 80ms/step - loss: 0.1774 - accura

##### Measure time of the default keras `flow_from_directory`

In [20]:
model = build_fresh_model(NUM_CLASSES)

In [21]:
start = time.perf_counter()
_ = model.fit(
    keras_data_gen, 
    steps_per_epoch=train_steps_per_epoch,    
    epochs=EPOCHS
    )

print(f"Train job took: {time.perf_counter() - start} seconds.")

Epoch 1/10
115/115 [==============================] - 15s 132ms/step - loss: 0.6864 - accuracy: 0.7520
Epoch 2/10
115/115 [==============================] - 15s 131ms/step - loss: 0.3790 - accuracy: 0.8676
Epoch 3/10
115/115 [==============================] - 15s 128ms/step - loss: 0.3143 - accuracy: 0.8951
Epoch 4/10
115/115 [==============================] - 15s 127ms/step - loss: 0.2700 - accuracy: 0.9054
Epoch 5/10
115/115 [==============================] - 15s 128ms/step - loss: 0.2522 - accuracy: 0.9134
Epoch 6/10
115/115 [==============================] - 15s 128ms/step - loss: 0.2267 - accuracy: 0.9229
Epoch 7/10
115/115 [==============================] - 15s 129ms/step - loss: 0.2125 - accuracy: 0.9278
Epoch 8/10
115/115 [==============================] - 15s 127ms/step - loss: 0.2001 - accuracy: 0.9332
Epoch 9/10
115/115 [==============================] - 15s 127ms/step - loss: 0.1869 - accuracy: 0.9403
Epoch 10/10
115/115 [==============================] - 15s 130ms/step - l

##### Measure time of the keras.preprocessing `image_dataset_from_directory` loader:

In [22]:
model = build_fresh_model(NUM_CLASSES)

In [23]:
start = time.perf_counter()
_ = model.fit(
    image_dataset_from_directory, 
    steps_per_epoch=train_steps_per_epoch,    
    epochs=EPOCHS
    )

print(f"Train job took: {time.perf_counter() - start} seconds.")

Epoch 1/10
115/115 [==============================] - 12s 102ms/step - loss: 0.7068 - accuracy: 0.7458
Epoch 2/10
115/115 [==============================] - 12s 106ms/step - loss: 0.3725 - accuracy: 0.8651
Epoch 3/10
115/115 [==============================] - 12s 104ms/step - loss: 0.3077 - accuracy: 0.8910
Epoch 4/10
115/115 [==============================] - 12s 108ms/step - loss: 0.2727 - accuracy: 0.9068
Epoch 5/10
115/115 [==============================] - 12s 102ms/step - loss: 0.2430 - accuracy: 0.9144
Epoch 6/10
115/115 [==============================] - 12s 101ms/step - loss: 0.2230 - accuracy: 0.9243
Epoch 7/10
115/115 [==============================] - 12s 100ms/step - loss: 0.2018 - accuracy: 0.9330
Epoch 8/10
115/115 [==============================] - 12s 100ms/step - loss: 0.1879 - accuracy: 0.9327
Epoch 9/10
115/115 [==============================] - 12s 101ms/step - loss: 0.1860 - accuracy: 0.9373
Epoch 10/10
115/115 [==============================] - 12s 102ms/step - l

### Conclusions

Using MobilenetV2 as feature extractor for a new classification problem, the time to perform the train job took respectively:
1. 152.09 seconds, when using default Keras loader
2. 127.19 seconds, when using keras.preprocessing `image_dataset_from_directory`
3. 94.29 seconds, when using `tfdata-image-loader` module

The option to use my implementation provided best results in the above use case.